In [1]:
import os
import pandas as pd
import numpy as np

import requests
import json
from pandas.io.json import json_normalize

In [2]:
path = r"/Users/jib/Documents/Projects/Hackathon"
os.chdir(path)
os.getcwd()

'/Users/jib/Documents/Projects/Hackathon'

# Library

### P' Bright part

In [140]:
hs_code = '8471'
url = "http://comtrade.un.org/api/get?r=" + "All" + "&freq=" + "A" + "&ps=" + "2014,2015,2016,2017,2018" +"&px="+ "HS" + "&p=" + "0,764" + "&rg=" + "1" + "&cc=" + hs_code + "&fmt=" + "json"  + "&max=" + "10000" + "&type=" + "C" 
url = requests.get(url)
data = json.loads(url.text)
    
df_im = json_normalize(data['dataset'])
    
# 1. import from world growth (CAGR)
df_im_w = df_im[df_im['pt3ISO'] == 'WLD']
df_im_w = df_im_w[['rt3ISO','period','TradeValue']]
    
pv = df_im_w.pivot_table(index=['rt3ISO'],columns=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
pv = pv['TradeValue'].reset_index()
    
pv['CAGR'] = pv.iloc[:, -1].div(pv.iloc[:, 1]).pow(1./(len(pv.columns) - 2)).sub(1)


##### Start here
test= pv.drop(columns = 'CAGR')

#pd.wide_to_long(test)#['tag_columns_min_not0'] = test['2014','2015','2016','2017','2018'].apply(idx.min, axis = 1)
ren_dict = {2014:'year_2014',2015:'year_2015',2016:'year_2016',2017:'year_2017',2018:'year_2018'
           }
test =test.rename(columns = ren_dict)

#test.columns.tolist()
test = pd.wide_to_long(test, stubnames = 'year_',i= ["rt3ISO"], j = 'year').reset_index()

yrmin = test[test['year_'] > 0 ].groupby('rt3ISO')['year'].min().reset_index().rename(columns = {'year':'yr_min'})
yrmax = test[test['year_'] > 0 ].groupby('rt3ISO')['year'].max().reset_index().rename(columns = {'year':'yr_max'})


test = test.merge(yrmin, how = 'left').merge(yrmax , how = "left").rename(columns  ={'year_':'value'})

test = test.sort_values(by =['rt3ISO',"year"])
min_ = test[test['year'] == test['yr_min']]

max_ = test[test['year'] == test['yr_max']]

min_ = min_[["rt3ISO","year","value"]].rename(columns = {'year':'year_min', 'value':'value_beg'})
max_ = max_[["rt3ISO","year","value"]].rename(columns = {'year':'year_max','value':'value_end'})
min_max = min_.merge(max_)   
min_max['CAGR'] = (min_max['value_end']/min_max['value_beg'])**(1/(min_max['year_max']  -min_max['year_min']    )) -1  
min_max['hwmanyyear'] = min_max['year_max'] - min_max['year_min']
min_max
# )
#test['CAGR'] = (test[test['year'] == year_max]['year_'].item()/ test[test['year'] == year_min]['year_'].item())**(1/(year_max - year_min)) - 1
#test

,rt3ISO,year_min,value_beg,year_max,value_end,CAGR,hwmanyyear
0,ABW,2014,1.032866e+07,2018,1.018480e+07,-0.003500,4
1,AGO,2014,1.449722e+08,2015,8.677315e+07,-0.401450,1
2,ALB,2014,7.740548e+06,2018,2.015834e+07,0.270342,4
3,AND,2014,2.397756e+07,2018,2.119314e+07,-0.030389,4
4,ARE,2014,8.573917e+09,2018,6.024237e+09,-0.084453,4
5,ARG,2014,6.664348e+08,2018,7.392039e+08,0.026246,4
6,ARM,2014,3.810943e+07,2018,5.420942e+07,0.092095,4
7,ATG,2014,4.758940e+06,2018,9.430804e+06,0.186478,4
8,AUS,2014,6.549597e+09,2018,7.321692e+09,0.028251,4
9,AUT,2014,1.860262e+09,2018,1.903525e+09,0.005764,4


In [58]:
def potential_ind(hs_code):
    p_index = pd.read_excel('Country dict.xls')
    p_index['hs_code'] = str(hs_code)
    
    df_pd  = product_ind(hs_code)
    p_index = p_index.merge(df_pd, left_on='CCISO3', right_on='CCISO3', how="left")
    
    df_cnty = country_ind()
    p_index = p_index.merge(df_cnty, left_on=['CCISO3'], right_on=['CCISO3'], how="left")
    
    return p_index
    
def product_ind(hs_code):
    url = "http://comtrade.un.org/api/get?r=" + "All" + "&freq=" + "A" + "&ps=" + "2014,2015,2016,2017,2018" +"&px="+ "HS" + "&p=" + "0,764" + "&rg=" + "1" + "&cc=" + hs_code + "&fmt=" + "json"  + "&max=" + "10000" + "&type=" + "C" 
    url = requests.get(url)
    data = json.loads(url.text)
    
    df_im = json_normalize(data['dataset'])
    
    # 1. import from world growth (CAGR)
    df_im_w = df_im[df_im['pt3ISO'] == 'WLD']
    df_im_w = df_im_w[['rt3ISO','period','TradeValue']]
    
    pv = df_im_w.pivot_table(index=['rt3ISO'],columns=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
    pv = pv['TradeValue'].reset_index()
    
    pv['CAGR'] = pv.iloc[:, -1].div(pv.iloc[:, 1]).pow(1./(len(pv.columns) - 2)).sub(1)
    
    
    # 2. import from world share (cnty_im/ world_im)
    pv_2 = df_im_w.pivot_table(index=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
    pv_2 = pv_2['TradeValue'].reset_index()
    pv['s_2018'] = pv.iloc[:, -2].div(pv_2.loc[4,'TradeValue'])
    
    
    # 3. import from Thai share (im_from_th / im_from_world)
   # df_im_th = df_im[df_im['pt3ISO'] == 'THA']
   # pv_3 = df_im_th.pivot_table(index=['period'],values=['TradeValue'],aggfunc='sum').fillna(0)
    
    # 4. avg tariff rate
    df_at = pd.read_csv('TC_MFN.csv')
    cn_list = pd.read_excel('Country dict.xls')
    df_tr = df_at.merge(cn_list, left_on='Reporter', right_on='ctyName', how="left")
    
    if hs_code[0] == '0':
        hs_n = hs_code[1:]
    else:
        hs_n = hs_code

    df_tr = df_tr[(df_tr['Product'] == int(hs_n))][['CCISO3','AVDuty average']].reset_index()
    pv = pv.merge(df_tr, left_on='rt3ISO', right_on='CCISO3', how="left")

    return pv[['CCISO3','CAGR','s_2018','AVDuty average']]

def country_ind():
    cn_ind = pd.read_csv('country_ind.csv')
    cn_ind = cn_ind.drop(['ctyName','CCISO2'], axis=1)
    
    return cn_ind

def normalized(series):
    mean = series.mean()
    std = series.std()
    zscore = (series - mean)/std
    return zscore

# Backup

In [3]:
#### weight on

#market potential (50%)
w_im_growth = 0.1
w_im_share = 0.1
w_GDP = 0.1

#trade convenient (30%)
w_avg_tariff = 0.1 

w_lpi = 0.1
w_costtoex = 0.1
w_distances = 0.1
w_fta = 0.1

#country risk (20%)
w_political = 0.1
w_commercial = 0.1
w_corruption = 0.1